 ## Import packages and modules

In [1]:
from datetime import date
from datetime import timedelta
import os
import datetime
import pandas as pd
import ee
import geemap
import os
import eeDatabase_coreMethods as eedb_cor
import eeDatabase_collectionMethods as eedb_col
import eeDatabase_collectionInfo as eedb_colinfo

# ee.Authenticate()
ee.Initialize(project = "dri-apps")

Map = geemap.Map()

## Define Parameters and Load datasets

In [4]:
# ------------------------------------- Define parameters -----------------------------------------------
# Define time period to export
start_date = datetime.datetime(2023, 1, 1)
end_date = datetime.datetime(2023, 2, 1)

# -------------------------------- Define input Image Collection ----------------------------------------
# Define input dataset
# See dictionary below for list of input datasets
in_ic_name = 'GridMET'

# Define variable from dataset
# See dictionary below for variables available for each dataset
var_name = 'precip'

# ------------------------------- Define input Feature Collection ---------------------------------------
# Define input path for Feature Collection
in_fc_path = 'projects/dri-apps/assets/blm-admin/blm-natl-admu-districtoffice-polygons'
in_fc = ee.FeatureCollection(in_fc_path)

# ------------------------------ Define mask, if applicable --------------------------------------------
# For BLM, we will apply mask to field offices, district offices, and state offices, but not to allotments
# Apply mask for ownership, landcover, or other variables. Must be binary mask.
mask = True
mask_path = 'projects/dri-apps/assets/blm-admin/blm-natl-admu-sma-binary'

['IDAHO_EPSCOR/GRIDMET']


### Define additional parameters required for functions and output properties below

In [ ]:
# Define land unit names
if(in_fc_path == 'projects/dri-apps/assets/blm-admin/blm-natl-grazing-allotment-polygons'):
    land_unit_long = 'BLM_Natl_Grazing_Allotment_Polygons'
    land_unit_short = 'BLM_Allotments'
    in_fc_id = 'ALLOT_ID'
elif(in_fc_path == 'projects/dri-apps/assets/blm-admin/blm-natl-admu-fieldoffice-polygons'):
    land_unit_long = 'BLM_Natl_FieldOffice_Polygons'
    land_unit_short = 'BLM_FieldOffices'
    in_fc_id = 'FO_ID'
elif(in_fc_path == 'projects/dri-apps/assets/blm-admin/blm-natl-admu-districtoffice-polygons'):
    land_unit_long = 'BLM_Natl_DistrictOffice_Polygons'
    land_unit_short = 'BLM_DistrictOffices'
    in_fc_id = 'DO_ID'
elif(in_fc_path == 'projects/dri-apps/assets/blm-admin/blm-natl-admu-stateoffice-polygons'):
    land_unit_long = 'BLM_Natl_StateOffice_Polygons'
    land_unit_short = 'BLM_StateOffices'
    in_fc_id = 'SO_ID'

# Pull out additional variables needed to run exports
in_ic_paths = eedb_colinfo.in_ic_dict.get(in_ic_name).get('in_ic_paths')
in_ic_res = ee.Number(ee.ImageCollection(in_ic_paths[0]).first().projection().nominalScale()).round().getInfo()
var_type = eedb_colinfo.in_ic_dict.get(in_ic_name).get('var_type')
var_units = eedb_colinfo.var_dict.get(var_name).get('units')
out_path = f"projects/dri-apps/assets/blm-database/{land_unit_short.replace('_', '').lower()}-{in_ic_name.replace('_', '').lower()}-{var_name.replace('_', '').lower()}"

## Create database image collection and append time-series images

In [3]:
# If there is no Image Collection asset at the out_path create one and export ID image
if os.system(f"earthengine asset info {out_path}") == 256:

    print("Initializing Image Collection by creating EE asset and exporting ID image")
    
    # Create dictionary of properties
    properties = {'system:index': '0_id', 'land_unit_long': land_unit_long, 'land_unit_short': land_unit_short, 'in_fc_path': in_fc_path, 
                  'in_fc_id': in_fc_id, 'in_ic_paths': in_ic_paths, 'in_ic_path': in_ic_paths[0], 'in_ic_name': in_ic_name, 'in_ic_res': in_ic_res, 'var_type': var_type, 
                  'var_name': var_name, 'var_units': var_units}
    if mask == True:
            properties['mask_path'] = mask_path
    elif mask == False:
            properties['mask_path'] = 'None'
    
    eedb_cor.initialize_collection(out_path = out_path, properties = properties)

    
# If there is an Image Collection asset at the out_path export time-series images
elif os.system(f"earthengine asset info {out_path}") == 0:

    print(f"Appending to Image Collection for dates {start_date} - {end_date}")

    # Get dates for image collection based on start and end date
    dates = eedb_cor.get_collection_dates(in_ic_paths = in_ic_paths, start_date = start_date, end_date = end_date)

    # Loop over selected dates to pre-process and export
    for date in dates:
        
        print("Running ", datetime.datetime.fromtimestamp(date/1000.0))
        
        # Parse date for ID
        date_ymd = datetime.datetime.fromtimestamp(date/1000.0).strftime('%Y%m%d')
        
        # Create dictionary of properties for image    
        properties = {'system:index': date_ymd, 'system:time_start': date, 'land_unit_long': land_unit_long, 'land_unit_short': land_unit_short, 'in_fc_path': in_fc_path,\
                      'in_fc_id': in_fc_id, 'in_ic_paths': in_ic_paths, 'in_ic_path': in_ic_paths[0], 'in_ic_name': in_ic_name, 'in_ic_res': in_ic_res, 'var_type': var_type,\
                      'var_name': var_name, 'var_units': var_units, 'mask': mask}
        
        eedb_cor.run_image_export(date = date, out_path = out_path, properties = properties)

{
  "id": "projects/dri-apps/assets/blm-database/blmdistrictoffices-gridmet-precip",
  "name": "projects/dri-apps/assets/blm-database/blmdistrictoffices-gridmet-precip",
  "type": "IMAGE_COLLECTION",
  "updateTime": "2023-06-15T17:48:48.889795Z"
}
{
  "id": "projects/dri-apps/assets/blm-database/blmdistrictoffices-gridmet-precip",
  "name": "projects/dri-apps/assets/blm-database/blmdistrictoffices-gridmet-precip",
  "type": "IMAGE_COLLECTION",
  "updateTime": "2023-06-15T17:48:48.889795Z"
}
Appending to Image Collection for dates 2023-01-01 00:00:00 - 2023-02-01 00:00:00
Running  2023-01-04 23:00:00
['IDAHO_EPSCOR/GRIDMET']
precip
1672898400000
['20230105']
['20230105']
Running  2023-01-09 23:00:00
['IDAHO_EPSCOR/GRIDMET']
precip
1673330400000
['20230110']
['20230110']
Running  2023-01-14 23:00:00
['IDAHO_EPSCOR/GRIDMET']
precip
1673762400000
['20230115']
['20230115']
Running  2023-01-19 23:00:00
['IDAHO_EPSCOR/GRIDMET']
precip
1674194400000
['20230120']
['20230120']
Running  2023-01-2

### Check completeness of image collections and re-run any images that are missing

In [66]:
# Get list of all dates
all_dates = eedb_col.get_collection_dates(in_ic_paths, start_date, end_date)

# Get list of dates from collection
coll_dates = ee.ImageCollection(out_path).aggregate_array('system:time_start').distinct().getInfo()

# Get list of dates missing from collection
miss_dates = set(all_dates) - set(coll_dates)
print("These dates are missing and will be rerun ", miss_dates)

for date in miss_dates:
    print("Running ", datetime.datetime.fromtimestamp(date/1000.0))
    
    # Parse date for ID
    date_ymd = datetime.datetime.fromtimestamp(date/1000.0).strftime('%Y%m%d')

    # Create dictionary of properties for image    
    properties = {'system:index': date_ymd, 'system:time_start': date, 'land_unit_long': land_unit_long, 'land_unit_short': land_unit_short, 'in_fc_path': in_fc_path,\
                'in_fc_id': in_fc_id, 'in_ic_paths': in_ic_paths, 'in_ic_path': in_ic_paths[0], 'in_ic_name': in_ic_name, 'in_ic_res': in_ic_res, 'var_type': var_type,\
                'var_name': var_name, 'var_units': var_units, 'mask': mask}
        
    eedb_cor.run_image_export(date = date, out_path = out_path, properties = properties)

These dates are missing and will be rerun  {646207200000, 644911200000}
Running  1990-06-24 00:00:00
Running  1990-06-09 00:00:00


In [49]:
# tasks = !geeadd tasks
task_count = int(tasks[1].replace("Tasks Pending: ", ""))
task_count
# dict(tasks).get('Tasks Running')

2325